In [1]:
!pip install evaluate transformers datasets rouge_score tqdm peft nltk --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 w

In [2]:
!pip install -U datasets --quiet

In [3]:
import random
import numpy as np
import torch
import nltk
import time


from datasets import load_dataset
import evaluate

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from peft import get_peft_model, LoraConfig, TaskType

2025-05-29 05:43:17.666803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748497397.849466      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748497397.907356      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Sử dụng thiết bị: {device}")

Sử dụng thiết bị: cuda


In [5]:
MODEL_NAME = "t5-small"
OUTPUT_DIR = "./t5_small_lora_summarization_results"
SAVED_MODEL_PATH = "./saved_models/t5_small_lora_summarization"


In [6]:
print(f"\n--- Downloading tokenizer: {MODEL_NAME} ---")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


--- Downloading tokenizer: t5-small ---


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q", "v","o","k"], # Common target modules for T5
    task_type=TaskType.SEQ_2_SEQ_LM )


model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
model.to(device)

trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=512, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=4, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=4, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
               

In [8]:
print("\n--- Downloading and preprocessing data ---")
dataset = load_dataset("cnn_dailymail", "3.0.0")
train_raw = dataset['train']
val_raw = dataset['validation']
test_raw = dataset['test']

max_input_length = 512
max_target_length = 128


def preprocess_function(examples):
    inputs = examples["article"]

    # Tokenize văn bản đầu vào cho Encoder
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_ds = train_raw.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
tokenized_val_ds = val_raw.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
tokenized_test_ds = test_raw.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])


print("Train size:", len(tokenized_train_ds))
print("Val size", len(tokenized_val_ds))
print("Test size", len(tokenized_test_ds))



--- Downloading and preprocessing data ---


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Train size: 287113
Val size 13368
Test size 11490


In [9]:
# --- 5. Data Collator and Metrics ---
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=3e-4,
    num_train_epochs=1,
    logging_steps=100,
    save_steps=1000,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    bf16=(device == "cuda"),
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    predict_with_generate=True,
    label_names=["labels"],
)

In [11]:
print("\n--- Start Training ---")
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

start_time_train = time.time()
trainer.train()
end_time_train = time.time()
train_duration = end_time_train - start_time_train
print(f"\nFinish training in: {train_duration:.2f} seconds")


--- Start Training ---


/tmp/ipykernel_19/2769680743.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.030500,1.830368,25.119300,11.950800,20.569700,23.646700,19.999776



Finish training in: 10476.20 seconds


In [12]:
val_metrics = trainer.evaluate(tokenized_val_ds)
print("Roug_Score on validation test", val_metrics)

test_metrics = trainer.evaluate(tokenized_test_ds)
print("Roug_Score on test test", test_metrics)

Roug_Score on validation test {'eval_loss': 1.8303682804107666, 'eval_rouge1': 25.1193, 'eval_rouge2': 11.9508, 'eval_rougeL': 20.5697, 'eval_rougeLsum': 23.6467, 'eval_gen_len': 19.999775583482943, 'eval_runtime': 1133.0265, 'eval_samples_per_second': 11.798, 'eval_steps_per_second': 2.95, 'epoch': 1.0}
Roug_Score on test test {'eval_loss': 1.8437703847885132, 'eval_rouge1': 25.4037, 'eval_rouge2': 12.0639, 'eval_rougeL': 20.8388, 'eval_rougeLsum': 23.8942, 'eval_gen_len': 20.0, 'eval_runtime': 978.7606, 'eval_samples_per_second': 11.739, 'eval_steps_per_second': 2.935, 'epoch': 1.0}


In [13]:
print(f"\n--- Save model at : {SAVED_MODEL_PATH} ---")
trainer.save_model(SAVED_MODEL_PATH)
tokenizer.save_pretrained(SAVED_MODEL_PATH)
print("Save Completed.")


--- Save model at : ./saved_models/t5_small_lora_summarization ---
Save Completed.
